In [38]:
import torch 
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

In [39]:
input_size = 28
time_steps = 28
num_layers = 2
hidden_size = 256
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 2

In [40]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*time_steps, num_classes)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [41]:
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [42]:
model = BRNN(input_size, hidden_size, num_layers, num_classes)
x = torch.rand(64, 1, 28, 28)
x = x.squeeze(1)
model(x).shape

torch.Size([64, 10])

In [43]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

In [44]:
x = torch.rand(64, 1, 28, 28)
x = x.squeeze(1)
print(x.shape)
model(x).shape

torch.Size([64, 28, 28])


torch.Size([64, 10])

In [45]:
train_dataset = datasets.MNIST(root='.', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='.', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### RNN

In [8]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [9]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.squeeze(1)        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

100%|██████████| 938/938 [02:28<00:00,  6.33it/s]


In [46]:
def check_acc(loader, model):
    if loader.dataset.train:
        print('checking the accuracy on training data')
    else: 
        print('checking the accuracy on test data')
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.squeeze(1)
            scores = model(x)
            
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        print('{}/{} with accuracy {}'.format(num_correct, num_samples, float(num_correct)/float(num_samples)))
    
    model.train()

In [14]:
check_acc(train_loader, model)

  0%|          | 2/938 [00:00<00:47, 19.78it/s]

checking the accuracy on training data


100%|██████████| 938/938 [00:44<00:00, 20.98it/s]

59223/60000 with accuracy 0.98705


In [15]:
check_acc(test_loader, model)

  1%|▏         | 2/157 [00:00<00:07, 19.73it/s]

checking the accuracy on test data


100%|██████████| 157/157 [00:07<00:00, 21.46it/s]

9860/10000 with accuracy 0.986



### Bidirectional LSTM

In [47]:
model = BRNN(input_size, hidden_size, num_layers, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [48]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.squeeze(1)
        
        scores = model(data)
        loss = criterion(scores, targets)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

100%|██████████| 938/938 [07:20<00:00,  2.13it/s]


In [49]:
check_acc(train_loader, model)

  0%|          | 1/938 [00:00<02:36,  5.99it/s]

checking the accuracy on training data


100%|██████████| 938/938 [02:23<00:00,  6.54it/s]

58655/60000 with accuracy 0.9775833333333334


In [50]:
check_acc(test_loader, model)

  1%|          | 1/157 [00:00<00:25,  6.16it/s]

checking the accuracy on test data


100%|██████████| 157/157 [00:23<00:00,  6.58it/s]

9765/10000 with accuracy 0.9765
